In [ ]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

# Operations on Sequences

**CS1302 Introduction to Computer Programming**
___

In [ ]:
import random
%reload_ext divewidgets

## Mutating a list

```{important}

For list (but not tuple), subscription and slicing can also be used as the target of an assignment operation to mutate the list.
```

In [ ]:
%%optlite -h 350
b = [*range(10)]  # aliasing
b[::2] = b[:5]
b[0:1] = b[:5]
b[::2] = b[:5]  # fails

OPTWidget(value=None, height=350, script='b = [*range(10)]  # aliasing\nb[::2] = b[:5]\nb[0:1] = b[:5]\nb[::2]…

Last assignment fails because `[::2]` with step size not equal to `1` is an *extended slice*, which can only be assigned to a list of equal size.

**What is the difference between mutation and aliasing?**

In the previous code:
- The first assignment `b = [*range(10)]` is aliasing, which gives the list the target name/identifier `b`.
- Other assignments such as `b[::2] = b[:5]` are mutations that [calls `__setitem__`](https://docs.python.org/3/reference/simple_stmts.html#assignment-statements) because the target `b[::2]` is not an identifier.

In [ ]:
list.__setitem__?

Signature:      list.__setitem__(self, key, value, /)
Call signature: list.__setitem__(*args, **kwargs)
Type:           wrapper_descriptor
String form:    <slot wrapper '__setitem__' of 'list' objects>
Namespace:      Python builtin
Docstring:      Set self[key] to value.


**Exercise** 

Explain why the check returns False.

In [ ]:
# %%optlite -l -h 400
a = b = [0]
b[0] = a[0] + 1
print(a[0] < b[0])

False


```{admonition} Solution

- The first line `a = b` makes `a` an alias of the same object `b` points to, and so
- the second line mutates the same object `a` and `b` point to.
- Hence, `a[0] == b[0]`.

```

**Exercise** 

Explain why the mutations below have different effects?

In [ ]:
a = [0, 1]
i = 0
a.__setitem__(i := i + 1, i)
print(a)

[0, 1]


In [ ]:
a = [0, 1]
i = 0
a[i := i + 1] = a[i]
print(a)

[0, 0]


```{admonition} Solution

`a[i := i + 1] = a[i]` is not the same as calling `a.__setitem__(i := i + 1, i)`. According to the [python documentation](https://docs.python.org/3/reference/simple_stmts.html#assignment-statements), 
- the expression to be assigned, i.e., `a[i]`, is first evaluated to `a[0]` and therefore `0`;
- since the target `a[i := i + 1]` is a user defined object, it continue to evaluate the target reference, i.e., the address of `a`, which corresponds to the list `[0, 1]`,
- followed by the subscription `i:=i+1`, which evaluates to `1`;
- Finally, `a.__setitem__` is called with the subscription, `1`, and expression to be assigned, `0`, and so the list `a` points to is mutuated to `[0, 1]`.

In comparison, directly calling `a.__setitem__(i := i + 1, i)`
- first evaluates the first argument `i := i + 1`, which gives `1` that is assigned to `i`, and
- then evaluates the second argument `i` to `1`, and so
- `a.__setitem__(1, 1)` is called instead, which does not change the list `a` points to.

```

**Why mutate a list?**

The following is another implementation of `composite_sequence` that takes advantage of the mutability of list. 

In [ ]:
%%optlite
def sieve_composite_sequence(stop):
    is_composite = [False] * stop  # initialization
    for factor in range(2, stop):
        if is_composite[factor]:
            continue
        for multiple in range(factor * 2, stop, factor):
            is_composite[multiple] = True
    return (x for x in range(4, stop) if is_composite[x])


for x in sieve_composite_sequence(100):
    print(x, end=" ")

OPTWidget(value=None, script='def sieve_composite_sequence(stop):\n    is_composite = [False] * stop  # initia…

The algorithm 
1. changes `is_composite[x]` from `False` to `True` if `x` is a multiple of a smaller number `factor`, and
2. returns a generator that generates composite numbers according to `is_composite`.

**Exercise** 

Is `sieve_composite_sequence` more efficient than your solution `composite_sequence`? Why?

In [ ]:
composite_sequence = lambda stop: (
    x for x in range(2, stop) if any(x % divisor == 0 for divisor in range(2, x))
)

In [ ]:
%%timeit
for x in composite_sequence(10000): pass

361 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
for x in sieve_composite_sequence(10000): pass

1.81 ms ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
for x in sieve_composite_sequence(10000000): pass

The line `if is_composite[factor]: continue` avoids the redundant computations of checking composite factors.

**Exercise** 

Note that the multiplication operation `*` is the most efficient way to [initialize a 1D list with a specified size](https://www.geeksforgeeks.org/python-which-is-faster-to-initialize-lists/), but we should not use it to initialize a 2D list. Fix the following code so that `a` becomes `[[1, 0], [0, 1]]`.

In [ ]:
%%optlite -h 300
a = [[0] * 2] * 2
a[0][0] = a[1][1] = 1
print(a)

OPTWidget(value=None, height=300, script='a = [[0] * 2] * 2\na[0][0] = a[1][1] = 1\nprint(a)\n')

In [ ]:
### BEGIN SOLUTION
a = [[0] * 2 for i in range(2)]
### END SOLUTION
a[0][0] = a[1][1] = 1
print(a)

[[1, 0], [0, 1]]


## Different methods to operate on a sequence

Recall the `quicksort` algorithm:

In [ ]:
def quicksort(seq):
    '''Return a sorted list of items from seq.'''
    if len(seq) <= 1:
        return list(seq)
    i = random.randint(0, len(seq) - 1)
    pivot, others = seq[i], [*seq[:i], *seq[i + 1:]]
    left = quicksort([x for x in others if x < pivot])
    right = quicksort([x for x in others if x >= pivot])
    return [*left, pivot, *right]


seq = [random.randint(0, 99) for i in range(10)]
print(seq, quicksort(seq), sep='\n')

[90, 56, 6, 61, 25, 3, 40, 19, 94, 54]
[3, 6, 19, 25, 40, 54, 56, 61, 90, 94]


There is also a built-in function `sorted` for sorting a sequence:

In [ ]:
sorted?
sorted(seq)

[3, 6, 19, 25, 40, 54, 56, 61, 90, 94]

Signature: sorted(iterable, /, *, key=None, reverse=False)
Docstring:
Return a new list containing all items from the iterable in ascending order.

A custom key function can be supplied to customize the sort order, and the
reverse flag can be set to request the result in descending order.
Type:      builtin_function_or_method


**Is `quicksort` quicker?**

In [ ]:
%%timeit
quicksort(seq)

12.1 µs ± 57.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
%%timeit
sorted(seq)

202 ns ± 0.228 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Python implements the [Timsort](https://en.wikipedia.org/wiki/Timsort) algorithm, which is very efficient.

**What are other operations on sequences?**

The following compares the lists of public attributes for `tuple` and `list`. 
- We determine membership using the [operator `in` or `not in`](https://docs.python.org/3/reference/expressions.html#membership-test-operations).
- Different from the [keyword `in` in a for loop](https://docs.python.org/3/reference/compound_stmts.html#the-for-statement), operator `in` calls the method `__contains__`.

In [ ]:
list_attributes = dir(list)
tuple_attributes = dir(tuple)

print(
    'Common attributes:', ', '.join([
        attr for attr in list_attributes
        if attr in tuple_attributes and attr[0] != '_'
    ]))

print(
    'Tuple-specific attributes:', ', '.join([
        attr for attr in tuple_attributes
        if attr not in list_attributes and attr[0] != '_'
    ]))

print(
    'List-specific attributes:', ', '.join([
        attr for attr in list_attributes
        if attr not in tuple_attributes and attr[0] != '_'
    ]))

Common attributes: count, index
Tuple-specific attributes: 
List-specific attributes: append, clear, copy, extend, insert, pop, remove, reverse, sort


- There are no public tuple-specific attributes, and
- all the list-specific attributes are methods that mutate the list, except `copy`.

The common attributes
- `count` method returns the number of occurrences of a value in a tuple/list, and
- `index` method returns the index of the first occurrence of a value in a tuple/list.

In [ ]:
%%optlite -l -h 450
a = (1,2,2,4,5)
count_of_2 = a.count(2)
index_of_1st_2 = a.index(2)

OPTWidget(value=None, height=450, script='a = (1,2,2,4,5)\ncount_of_2 = a.count(2)\nindex_of_1st_2 = a.index(2…

`reverse` method reverses the list instead of returning a reversed list.

In [ ]:
%%optlite -h 300
a = [*range(10)]
print(reversed(a))
print(*reversed(a))
print(a.reverse())

OPTWidget(value=None, height=300, script='a = [*range(10)]\nprint(reversed(a))\nprint(*reversed(a))\nprint(a.r…

- `copy` method returns a copy of a list.  
- `tuple` does not have the `copy` method but it is easy to create a copy by slicing.

In [ ]:
%%optlite -h 400
a = [*range(10)]
b = tuple(a)
a_reversed = a.copy()
a_reversed.reverse()
b_reversed = b[::-1]

OPTWidget(value=None, height=400, script='a = [*range(10)]\nb = tuple(a)\na_reversed = a.copy()\na_reversed.re…

`sort` method sorts the list *in place* instead of returning a sorted list.

In [ ]:
%%optlite -h 300
import random
a = [random.randint(0,10) for i in range(10)]
print(sorted(a))
print(a.sort())

OPTWidget(value=None, height=300, script='import random\na = [random.randint(0,10) for i in range(10)]\nprint(…

- `extend` method that extends a list instead of creating a new concatenated list.
- `append` method adds an object to the end of a list.
- `insert` method insert an object to a specified location.

In [ ]:
%%optlite -h 300
a = b = [*range(5)]
print(a + b)
print(a.extend(b))
print(a.append('stop'))
print(a.insert(0,'start'))

OPTWidget(value=None, height=300, script="a = b = [*range(5)]\nprint(a + b)\nprint(a.extend(b))\nprint(a.appen…

- `pop` method deletes and return the last item of the list.  
- `remove` method removes the first occurrence of a value in the list.  
- `clear` method clears the entire list.

We can also use the function `del` to delete a selection of a list.

In [ ]:
%%optlite -h 300
a = [*range(10)]
del a[::2]
print(a.pop())
print(a.remove(5))
print(a.clear())

OPTWidget(value=None, height=300, script='a = [*range(10)]\ndel a[::2]\nprint(a.pop())\nprint(a.remove(5))\npr…